In [54]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
from collections import deque
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import random
import glob
from IPython.display import Video
from tqdm import tqdm

In [55]:
class QNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(state_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, action_dim)
        )

    def forward(self, x):
        return self.model(x)

In [56]:
target_net = QNetwork(8, 4)

In [57]:
target_net.load_state_dict(torch.load("lunarlander-wind-3k.pth"))

/tmp/ipykernel_14185/2506464939.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  target_net.load_state_dict(torch.load("lunarlander-wind-3k.pth"))


<All keys matched successfully>

In [58]:
mean_rewards = 0

In [59]:
testing = gym.make("LunarLander-v3", render_mode="rgb_array",gravity=-10.0, enable_wind=True, wind_power=15.0, turbulence_power=1.5)
testing = RecordVideo(testing, video_folder="video_folder",episode_trigger=lambda ep: ep % 50 == 0, name_prefix="lunar_lander", disable_logger=True)
total_success = 0
for _ in tqdm(range(500)):
    state, _ = testing.reset()
    done = False
    total_reward = 0
    while not done:
        action = target_net(torch.FloatTensor(state).unsqueeze(0)).argmax().item()
        next_state, reward, terminated, truncated, _ = testing.step(action)
        total_reward += reward
        done = terminated or truncated
        state = next_state
        testing.render()

    if(total_reward >= 200):
        total_success += 1
    mean_rewards += total_reward
mean_rewards /= 500
print(f"Mean reward: {mean_rewards}")
print(f"Total done: {total_success}/500, percentage: {total_success/500*100}%")

testing.close()

100%|██████████| 500/500 [07:08<00:00,  1.17it/s]

Mean reward: 225.29876652556752
Total done: 393/500, percentage: 78.60000000000001%
